<a href="https://colab.research.google.com/github/astridmo/SmartChargingEV/blob/Add-optimization/Masteroppgave_automatisert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimerer lading av elbilflåte

Velg input

# Input

__Mulige input:__

year: ["2019", "2020", "2021", "2023"]

Power tariff: ["Ingen", "Reell", "HoyereTredelt", "LavereTredelt", "Hoyest"]

Effekt = ["Reell", "22"]

In [ ]:
#year_list = ["2019", "2021", "2023", "2030"]
year_list = ["2023", "2022"]
#peak_tariff_list = ["Reell", "HoyereTredelt", "LavereTredelt", "Hoyest", "Ingen"]
peak_tariff_list = ["Lik"]
effekt_list = ["Reell"]

save_json = True

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
#year_list = ["2020"]
#peak_tariff_list = ["Ingen", "Reell"]
#effekt_list = ["22"]
#save_json = True

# Kode

## Initialize

In [ ]:
# ==========================================
# Kontroller input
# ==========================================

year_full_list = ["2019", "2020", "2021", "2023", "2030", "2022"]
peak_tariff_full_list = ["Ingen", "Reell", "HoyereTredelt", "LavereTredelt", "Hoyest", "Lik"]
effekt_full_list = ["Reell", "22"]

for element in year_list:
    if element not in year_full_list:
        raise ValueError(f"Året {element} er ikke i definert year_full_list")

for element in peak_tariff_list:
    if element not in peak_tariff_full_list:
        raise ValueError(f"Effekttariffen {element} er ikke definert peak_tariff_full_list")

for element in effekt_list:
    if element not in effekt_full_list:
        raise ValueError(f"Effekten {element} er ikke definert i effekt_full_list")

In [ ]:
!python -m pip install -i https://pypi.gurobi.com gurobipy
!python -m pip install -U kaleido

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Looking in indexes: https://pypi.gurobi.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 14.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from gurobipy import *
import pandas as pd
import plotly.express as px
import json
from collections import defaultdict
from datetime import timedelta, datetime
import numpy as np
from google.colab import drive
import plotly.graph_objects as go

In [ ]:
drive.mount('/content/drive')


params = {
"WLSACCESSID": "536d1971-f9d1-462f-a749-e70576255d06",
"WLSSECRET": "42cd8734-42f2-42fc-b30c-89fa44364e19",
"LICENSEID": 2427574,
}

#import gurobipy as gp  # import the installed package
env = Env(params=params)

Mounted at /content/drive
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2427574
Academic license 2427574 - for non-commercial use only - registered to as___@nmbu.no


In [ ]:
# ===================================
# Power tariffs
# ===================================
peak_tariff_Reell = {
    1: 75, 2: 75, 3: 75,  # Vinter
    4: 32, 5: 32, 6: 32,  # Sommer
    7: 32, 8: 32, 9: 32,  # Sommer
    10: 75, 11: 75, 12: 75 # Vinter
}

peak_tariff_HoyereTredelt = {
    1: 150, 2: 150, 3: 80,  # Vinter
    4: 23, 5: 23, 6: 23,  # Sommer
    7: 23, 8: 23, 9: 23,  # Sommer
    10: 23, 11: 80, 12: 150 # Vinter
}

peak_tariff_LavereTredelt = {
    1: 125, 2: 125, 3: 65,  # Vinter
    4: 10, 5: 10, 6: 10,  # Sommer
    7: 10, 8: 10, 9: 10,  # Sommer
    10: 10, 11: 65, 12: 125 # Vinter
}

peak_tariff_Ingen = {
    1: 0, 2: 0, 3: 0,  # Vinter
    4: 0, 5: 0, 6: 0,  # Sommer
    7: 0, 8: 0, 9: 0,  # Sommer
    10: 0, 11: 0, 12: 0 # Vinter
}

peak_tariff_Hoyest = {
    1: 300, 2: 300, 3: 300,  # Vinter
    4: 75, 5: 75, 6: 75,  # Sommer
    7: 75, 8: 75, 9: 75,  # Sommer
    10: 300, 11: 300, 12: 300 # Vinter
}

peak_tariff_Lik = {
    1: 32, 2: 32, 3: 32,  # Vinter
    4: 32, 5: 32, 6: 32,  # Sommer
    7: 32, 8: 32, 9: 32,  # Sommer
    10: 32, 11: 32, 12: 32 # Vinter
}

# Oppretter en ordbok for å mappe navn til de tilsvarende ordbøkene
tariff_mapping = {
    "Ingen": peak_tariff_Ingen,
    "Hoyest": peak_tariff_Hoyest,
    "LavereTredelt": peak_tariff_LavereTredelt,
    "HoyereTredelt": peak_tariff_HoyereTredelt,
    "Reell": peak_tariff_Reell,
    "Lik": peak_tariff_Lik
}

# =================================================
# Files
# =================================================
df_spotprice_csv = 'spot_prices_2023-11-28.csv'
df_overview_csv_jan_mai = 'overview_wo_etterlading_januar_mai_2023-11-23.csv'
df_overview_csv_jun_okt = 'overview_wo_etterlading_juni_oktober_2023-11-23.csv'
df_detailed_csv = 'detailed_wo_etterlading_jan_okt_2023-11-23.csv'  # Også uten boundry mai/juni-biler
df_overview_csv = 'overview_wo_etterlading_jan_okt_2023-11-24.csv'  # Også uten boundry mai/juni-biler

## Funksjoner

### Simuleringsmodell


In [ ]:
def run_simulation(start_time_utc, peak_tariff, df_overview_csv, df_spotprice_csv, power_method):
    """Function to run the simulation"""

    df_spotprice = get_relevant_spotprices(df_spotprice_csv, start_time_utc)

    df_overview = get_df_overview(df_overview_csv)

    # Run optimization
    optimization_results = optimize_charging(df_overview, df_spotprice, peak_tariff, start_time_utc, power_method)



    return optimization_results


def get_relevant_spotprices(df_spotprice_csv, start_time_utc):
    """
    Function to return the spotprice-dataframe starting with the chosen start time
    :param df_spotprice_csv: DataFram of spotprices
    :param start_time_utc: The chosen start time (in UTC)
    :return: DataFrame of spotprices, starting whith the chosen start time
    """
    #start_time = pd.to_datetime(start_time_utc)
    df_spotprice = pd.read_csv(df_spotprice_csv, index_col="DateTimeUtc")  # Load csv
    df_spotprice.index = pd.to_datetime(df_spotprice.index) # Change index to datatime dtype

    # Make the DataFrame start with the chosen start time
    if start_time_utc in df_spotprice.index:
        df_spotprice = df_spotprice[~(df_spotprice.index < start_time_utc)]  # Delete all rows before start_time_utc
    else:
        raise ValueError('There is no spot price data available for the chosen dates')
    return df_spotprice


def get_df_overview(df_overview_csv):
    # Load files
    df_overview = pd.read_csv(df_overview_csv)

    # Change DateTime-columns to pandas DateTime
    df_overview["StartDateTime"] = pd.to_datetime(df_overview["StartDateTime"])
    df_overview["EndDateTime"] = pd.to_datetime(df_overview["EndDateTime"])

    # To get fewer cars
    #df_overview_simulation = df_overview[:200]
    df_overview_simulation = df_overview.copy()  # All cars

    if len(df_overview_simulation) != len(df_overview):  # Hvis man ikke studerer alle bilene
        df_overview_simulation = _new_starthour(df_overview_simulation)

    return df_overview_simulation


def _new_starthour(df_overview_simulation):
    """
    Denne funksjonen gjør kun en endring på original dataframe dersom ikke alle bilene blir studert.

    Funksjonen gjør at man starter på StartHour = 0.
    :param df_overview_simulation: DataFrame
    :return: df_overview der den første bilen har StartHour = 0.
    """
    # Number of hours between the departure of the last car and the arrival of car 1
    start_time = df_overview_simulation["StartDateTime"].min()  # The oldest datetime (start time for the first car)

    # Lag ankomsttimer til int, der den første timen er int 0. Timen etter dette er int 1.
    df_overview_simulation['StartHour'] = (df_overview_simulation.loc[:,'StartDateTime'] - start_time) // pd.Timedelta('1H')
    # Gjør også dette med avreisetidspunkt ved å ta utgangspunkt i int 0 fra StartHour
    df_overview_simulation['EndHour'] = (df_overview_simulation.loc[:,'EndDateTime'] - start_time) // pd.Timedelta('1H')

    df_overview_simulation = df_overview_simulation.reset_index(drop=True)

    print("Den siste timen er: ", df_overview_simulation["EndDateTime"].max())
    return df_overview_simulation


def optimize_charging(df_overview, df_spotprice, peak_costs, start_time_utc, power_method):
    # Antall elbiler
    N_cars = len(df_overview)

    # Number of hours between the departure of the last car and the arrival of car 1
    n_hours = int(df_overview["EndHour"].max())

    # Generer en tidsperiode fra start til slutt per time
    start_time_utc = pd.to_datetime(start_time_utc)
    print(n_hours)
    print(type(n_hours))
    print(timedelta(hours=n_hours))
    print(f"start_time {start_time_utc} |, timedelta: {timedelta(hours=n_hours)}, dtype:")
    all_dates = pd.date_range(start=start_time_utc, end=start_time_utc + timedelta(hours=n_hours), freq='H')

    # Lag en mapping fra hver time til tilsvarende måned
    hours_to_month = {hour: date.month for hour, date in enumerate(all_dates)}

    unique_months = set(hours_to_month.values())  # Hent de unike månedene


    class ElectricVehicle:
        def __init__(self, arrival_time, departure_time, required_charge, max_charge_rate):
            self.arrival_time = arrival_time
            self.departure_time = departure_time
            self.required_charge = required_charge
            self.max_charge_rate = max_charge_rate


    # Henter inn detaljer om bilene
    arrival_time = df_overview['StartHour']
    departure_time = df_overview['EndHour']
    required_charge = df_overview['Energy']
    #max_charge_rate_per_vehicle = 16  # Denne må forandres senere for å være mer virkelighetsnær
    #max_charge_rate_per_vehicle = np.minimum(df_overview['Max_power [kW]'] + 5, 22.5)
    if power_method == "Reell":
      max_charge_rate_per_vehicle = df_overview['Max_power [kW]']
      # Make the vehicles
      vehicles = [ElectricVehicle(arrival_time[i], departure_time[i], required_charge[i], max_charge_rate_per_vehicle[i]) for i in range(N_cars)]

    elif power_method == "22":
      max_charge_rate_per_vehicle = 22.1
      # Make the vehicles
      vehicles = [ElectricVehicle(arrival_time[i], departure_time[i], required_charge[i], max_charge_rate_per_vehicle) for i in range(N_cars)]

    #print(vehicles[v].max_charge_rate for v in range(N_cars))

    # Other stuff
    charging_cost_per_hour = df_spotprice["Price [NOK/MWh]"] / 1000  # Make to NOK/kWh
    charging_cost_per_hour = charging_cost_per_hour.iloc[:n_hours].tolist()
    max_total_load = 500

    # Create a new model
    m = Model(env=env)

    # Decision variables
    charge_rate = m.addVars(N_cars, n_hours, lb=0, vtype=GRB.CONTINUOUS, name="charge_rate")
    peak_charge = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="peak_charge")
    peak_load_monthly = m.addVars(unique_months, lb=0, vtype=GRB.CONTINUOUS, name="peak_load_monthly")

    # Objective function
    energy_cost = quicksum(charge_rate[v, t] * charging_cost_per_hour[t-1] for v in range(N_cars) for t in range(n_hours))
    peak_cost = quicksum(peak_costs[m] * peak_load_monthly[m] for m in unique_months)

    m.setObjective(energy_cost + peak_cost, GRB.MINIMIZE)
    con_power = []

    # Constraints
    for v in range(N_cars):
        # Constraint: Lad kun opp til den nødvendige mengden før avreise
        m.addConstr(
            quicksum(charge_rate[v, t] for t in range(vehicles[v].arrival_time, vehicles[v].departure_time)) == vehicles[v].required_charge,
            name=f"charge_constraint_{v}"
        )

        for t in range(n_hours):
            # Constraint: Ikke lad utenfor tiden som bilen står parkert
            if t < vehicles[v].arrival_time or t >= vehicles[v].departure_time:
                m.addConstr(charge_rate[v, t] == 0, name=f"no_charge_outside_stay_{v}_{t}")
            # Constraint: Ikke overskrid maksimal ladeeffekt per bil
            con_power.append(m.addConstr(charge_rate[v, t] <= vehicles[v].max_charge_rate, name=f"max_charge_rate_per_vehicle_{v}_{t}"))

    for t in range(n_hours):
        # Constraint: Ensuring the total charging at any time does not exceed the monthly peak for the corresponding month
        m.addConstr(
            quicksum(charge_rate[v, t] for v in range(N_cars)) <= peak_load_monthly[hours_to_month[t]],
            name=f"monthly_peak_{t}"
        )

        # Constraint: Ikke overskrid total ladekapasitet
        m.addConstr(
            quicksum(charge_rate[v, t] for v in range(N_cars)) <= max_total_load,
            name=f"capacity_limit"
        )

    # Sett DualReductions-parameteren til 0
    m.setParam('DualReductions', 0)
    m.setParam('Method', 0)  # For dual simplex


    torelax = con_power
    conpens = [0.1]*len(torelax)
    if power_method == "Reell":
      m.feasRelax(relaxobjtype=0, minrelax=True, vars=None, lbpen=None, ubpen=None, constrs=torelax, rhspen=conpens)

    # Optimize model
    m.optimize()

    # Display results

    if m.status == GRB.OPTIMAL:
        # Fortsett med å hente løsningsverdier
        pass
    elif m.status == GRB.INFEASIBLE:
        print("Modellen er uoverkommelig: ingen løsning funnet.")
        # Du kan bruke m.computeIIS() for å identifisere årsaken
    else:
        print(f"Optimalisering ble stoppet med status {m.status}")

    #for v in range(N_cars):
        #print(f"Vehicle {v+1} charging schedule: {[charge_rate[v, t].x for t in range(1, n_hours)]}")
        #individual_energy_cost = sum(charge_rate[v, t].x * charging_cost_per_hour[t-1] for t in range(n_hours))
        #print(f"Vehicle {v+1}  --> Energikostnad: {round(individual_energy_cost, 2)} kr")

    print("****************************")
    print("Objektiv-verdien gir: ", round(m.objVal, 2), "kr")
    print("Den totale kostnaden blir:", round(energy_cost.getValue() + peak_cost.getValue(), 2), "kr")

    # For energikostnaden må vi beregne den manuelt igjen ettersom den ikke er en egen variabel i modellen
    # total_energy_cost = sum(charge_rate[v, t].x * charging_cost_per_hour[t-1] for v in range(N_cars) for t in range(n_hours))
    print("Energikostnaden blir:", round(energy_cost.getValue(), 2), "kr")

    # Tilsvarende må vi beregne topplastkostnaden manuelt
    print("Topplastkostnaden blir:", round(peak_cost.getValue(), 2), "kr")

    # Månedlig peak load
    monthly_peak_loads = [peak_load_monthly[m].x for m in unique_months]
    print("Månedlig", monthly_peak_loads)



    def _get_optimization_results(N_cars, model, energy_cost, peak_cost, peak_load_monthly, charge_rate, charging_cost_per_hour):
        """
        Function to attrieve the relevant information from the optimizatin model and save for further analysis.

        :param model: The optimization model
        :param energy_cost: The energy cost (coming from the spot price)
        :param peak_cost: The peak cost (coming from the peak tariff)
        :param peak_load_monthly: The monthly peak loads
        :param charge_rate:
        :param charging_cost_per_hour:
        :return:
        """
        # Get the total load
        total_load_profile = [sum(charge_rate[v, t].X for v in range(N_cars)) for t in range(n_hours)]

        # Get the total costs
        exceeded_power = round(model.objVal,2)  # Objektivverdien gir hvor mye man har overskridet effekt-begrensningen for biler
        total_energy_cost = round(energy_cost.getValue(),2)  # Energy costs
        total_peak_cost = round(peak_cost.getValue(),2)  # Peak costs
        total_cost = total_energy_cost + total_peak_cost # Total cost = energy cost + peak cost

        # Get peak load and peak cost of each month
        peak_load_monthly_value = [peak_load_monthly[m].x for m in unique_months]
        peak_load_monthly_value = dict(zip(unique_months, peak_load_monthly_value))  #  Make a dict with month as key and load as value

        peak_cost_monthly = [peak_tariff[m] * peak_load_monthly_value[m] for m in unique_months]
        peak_cost_monthly = dict(zip(unique_months, peak_cost_monthly))  #  Make a dict with month as key and peak cost as value

        # The energy cost of each car
        vehicle_energy_cost = [sum(charge_rate[v, t].x * charging_cost_per_hour[t-1] for t in range(n_hours))for v in range(N_cars)]

        # =================================
        # Get
        # =================================
        vehicles_charging_schedule = {}  # The charging schedule for each car
        vehicle_total_charge = []  # The total charged energy for each car

        for v in range(N_cars):
            vehicle_charge_rates = [charge_rate[v, t].x for t in range(n_hours)]  # Todo: Denne har du også regnet ut ovenfor..
            vehicle_total_charge.append(sum(vehicle_charge_rates))

            vehicles_charging_schedule[v] = vehicle_charge_rates


        # Get monthly energy cost
        hours_to_month_copy = hours_to_month.copy()
        hours_to_month_copy.pop(list(hours_to_month_copy.keys())[-1])

        monthly_energy_charge = defaultdict(int)
        monthly_energy_cost = defaultdict(int)

        # Legg til energiforbruket  og energikostnaden for den gitte timen til den tilsvarende måneden
        for key, month in hours_to_month_copy.items():
            monthly_energy_charge[month] += total_load_profile[key]
            monthly_energy_cost[month] += total_load_profile[key] * charging_cost_per_hour[key]


        # Make dict of all the results
        export_dict = {"peak_tariff": peak_tariff, "start_date": start_time_utc.strftime('%Y-%m-%d'),
                       "total_cost": total_cost, "total_energy_cost": total_energy_cost,
                       "total_peak_cost": total_peak_cost, "exceeded_power": exceeded_power,
                       "peak_load_monthly": peak_load_monthly_value,
                       "peak_cost_monthly": peak_cost_monthly, "vehicle_energy_cost": vehicle_energy_cost,
                       "total_load_profile": total_load_profile, "vehicles_charging_schedule": vehicles_charging_schedule,
                       "monthly_energy_charge": monthly_energy_charge, "monthly_energy_cost": monthly_energy_cost}

        return export_dict

    # ========================================================
    # Save the relevant information from the optimization
    # ========================================================

    optimization_results = _get_optimization_results(N_cars, m, energy_cost, peak_cost, peak_load_monthly, charge_rate, charging_cost_per_hour)

    return optimization_results

### Virkelig lastprofil

In [ ]:
def get_real_costs(df_overview_csv, df_detailed_csv, df_spot_prices_csv, peak_cost_dict, start_time):
    """
    Funksjon for å beregne kostnadene ved reell lastprofil ved ønsket starttidspunkt.
    For å gjøre dette blir starttidspunktet i df_detailed og df_overview forandret til 'start_time'

    :param df_overview: DataFrame med EV-data på oversiktsformat
    :param df_detailed: DataFrame med EV-data på detaljert format
    :param peak_cost_dict: dict av effekttariffer
    :param start_time: Ønsket starttidspunkt for simuleringen
    :return: total_monthly_costs (DataFrame med månedlige kostnader), df_overview_shifted (flyttet df_overview til ønsket starttidspunkt), df_detailed_shifted (flyttet df_detailed til ønsket starttidspunkt)
    """
    # Last inn filer
    df_detailed = pd.read_csv(df_detailed_csv)
    df_spotprice = pd.read_csv(df_spotprice_csv, index_col="DateTimeUtc")  # Load csv
    df_overview = get_df_overview(df_overview_csv)
    df_detailed["DateTimeUtc"] = pd.to_datetime(df_detailed["DateTimeUtc"]) # Change DateTime-columns to pandas DateTime
    df_spotprice.index = pd.to_datetime(df_spotprice.index) # Change index to datatime dtype
    start_time = pd.to_datetime(start_time)

    # Flytt starttidspunkt til ønsket starttid for simuleringen
    df_overview_shifted, df_detailed_shifted = shift_dataframe_time(
        df_overview, df_detailed, start_time)

    # Legg til riktig energikostnad i df_overview og df_detailed
    df_detailed_shifted, df_overview_shifted = add_energy_cost(
        df_overview_shifted, df_detailed_shifted, df_spotprice)

    # Get load profile
    load_df = df_detailed_shifted.groupby(pd.Grouper(key='DateTimeUtc', freq='H'))[['Charged_energy', 'energy_cost']].sum()

    # Finn månedlige totale kostnader (både effektkostnader og strømkostnader)
    total_monthly_costs = calculate_monthly_costs(df_detailed_shifted, load_df, peak_cost_dict)

    return total_monthly_costs, df_overview_shifted, df_detailed_shifted, load_df


def shift_dataframe_time(df_overview, df_detailed, time_string):
    """
    Forskyver tidspunktene i en spesifikk kolonne av en DataFrame basert på forskjellen
    mellom en gitt tid og minimumsverdien i den kolonnen.

    :param df_overview: pandas DataFrame som inneholder overordnet oversikt over ladehistorikk
    :param df_detailed: pandas DataFrame som inneholder kolonnen med tidspunktene
    (og detaljert ladehistorikk)
    :param time_string: Tidspunkt gitt som en streng. Dette er starttidspunktet for simuleringen.
    :return: En ny DataFrame med tidspunktene forskjøvet.
    """

    # Sikre at kolonnen er i datetime-format
    df_detailed["DateTimeUtc"] = pd.to_datetime(df_detailed["DateTimeUtc"])

    # Finn og rund ned minimumsverdien i angitt kolonne til nærmeste time
    min_time = df_detailed["DateTimeUtc"].min().floor('H')

    # Konverter streng til datetime
    specified_time = pd.to_datetime(time_string)

    # Beregn forskjellen i timer
    hours_to_shift = (specified_time - min_time).total_seconds() / 3600

    # Forskyv tidspunktene i df_detailed
    df_detailed_shifted = df_detailed.copy()
    df_detailed_shifted["DateTimeUtc"] = df_detailed_shifted["DateTimeUtc"] + pd.Timedelta(hours=int(hours_to_shift))

    # Forskyv tidspunktene i df_overview
    df_overview_shifted = df_overview.copy()
    df_overview_shifted[['StartDateTime', 'EndDateTime']] = df_overview_shifted[['StartDateTime', 'EndDateTime']].apply(lambda x: x + pd.Timedelta(hours=int(hours_to_shift)))

    return df_overview_shifted, df_detailed_shifted


def add_energy_cost(df_overview, df_detailed, df_spot_prices):
    """
    Legger til strømkostnaden til df_detailed og df_overview vha priser fra df_spot_prices.
    """
    df_detailed = df_detailed.reset_index()
    # ===================================================
    # Add energy cost for each timestamp in df_detailed
    # ===================================================
    for k in range(len(df_detailed)):
        # Round down to the closest hour. The spot price is pr hour.
        rounded_datetime = df_detailed.loc[df_detailed.index[k], "DateTimeUtc"].floor("H")
        # Find spot price for the given hour
        spot_price = df_spot_prices.loc[rounded_datetime, "Price [NOK/MWh]"]

        df_detailed.loc[df_detailed.index[k], "energy_cost"] = spot_price * df_detailed.loc[df_detailed.index[k], "Charged_energy"] / 1000  # Add energy cost

    # =============================================================
    # Add total energy cost for each charging cycle in df_overview
    # =============================================================
    for cycle_id in df_detailed["charge_cycle_id"].unique():
        total_cost = df_detailed.loc[df_detailed["charge_cycle_id"] == cycle_id, 'energy_cost'].sum()
        df_overview.loc[df_overview["charge_cycle_id"] == cycle_id, "energy_cost"] = total_cost

    return df_detailed, df_overview


def calculate_monthly_costs(df_detailed, load_df, peak_cost_dict):
    # Grupper dataene etter år og måned, og finn indeksen for maksimalt forbruk i hver gruppe
    idx = load_df.groupby([load_df.index.month])['Charged_energy'].idxmax()

    # Bruk indeksene fra forrige trinn til å hente de relevante radene fra den originale DataFrame
    highest_per_month = load_df.loc[idx]

    highest_per_month["Month"] = idx.index
    highest_per_month = highest_per_month.sort_index()

    df_peak_cost = highest_per_month[["Month", "Charged_energy"]]

    df_peak_cost['Monthly_Peak_Cost'] = df_peak_cost['Month'].map(peak_cost_dict) * df_peak_cost["Charged_energy"]

    df_peak_cost = df_peak_cost.rename(columns={"Charged_energy": "Peak_load"})

    total_monthly_cost = df_detailed.groupby(pd.Grouper(key='DateTimeUtc', freq='M'))[['Charged_energy', 'energy_cost']].sum()

    total_monthly_cost["Month"] = total_monthly_cost.index.month

    # Slår sammen df_peak_cost og total_monthly_cost basert på 'Month' kolonnen
    total_monthly_cost = pd.merge(df_peak_cost, total_monthly_cost, on="Month")

    total_monthly_cost.set_index('Month', inplace=True)

    # Total kostnad
    total_monthly_cost["total_cost"] = total_monthly_cost["Monthly_Peak_Cost"] + total_monthly_cost["energy_cost"]

    return total_monthly_cost

###Figurer

In [ ]:
def make_figures(optimization_results, real_results, start_time_utc, months):
  """Funksjon for å lage alle figurene"""

  months = ['januar', 'februar', 'mars', 'april', 'mai', 'juni', 'juli', 'august', 'september', 'oktober']

  fig_opt_load_profile, total_load_df = make_fig_opt_load_profile(optimization_results, start_time_utc)
  print("opt load profile ok")
  fig_load_duration = make_fig_load_duration(total_load_df, real_results)
  print("load duration ok")
  fig_monthly_cost = make_mnd_kostnadsfordeling(months, optimization_results, real_results)
  print("monthly cost ok")
  fig_total_cost = make_total_kostnadsfordeling(optimization_results, real_results)
  print("total cost ok")
  fig_maks_effekt = make_fig_maks_effekt(optimization_results, real_results, months)
  print("maks effekt ok")
  #fig_violin_effektfordeling = make_violin_effektfordeling(optimization_results, real_results, months, start_time_utc)
  print("violin ok")
  fig_max_power = make_fig_max_power(optimization_results)
  print("max power ok")

  fig_dict = {
    'fig_opt_load_profile': fig_opt_load_profile,
    'fig_load_duration': fig_load_duration,
    'fig_monthly_cost': fig_monthly_cost,
    'fig_total_cost': fig_total_cost,
    'fig_maks_effekt': fig_maks_effekt,
    #'fig_effektfordeling': fig_violin_effektfordeling,
    'fig_max_power': fig_max_power
  }

  return fig_dict

def make_fig_opt_load_profile(optimization_results, start_time_utc):
  """Fuksjon for å lage lastkurve for optimalisert lading"""
  # Opprette en datetime index som starter fra start_time_utc og går hver time
  index = pd.date_range(start=start_time_utc, periods=len(optimization_results['total_load_profile']), freq='H')

  # Opprette DataFrame med den gitt listen som kolonne
  total_load_df = pd.DataFrame(optimization_results['total_load_profile'], index=index, columns=['optimized_load_profile'])

  fig_opt_load_profile = px.line(x=total_load_df.index, y=total_load_df['optimized_load_profile'], title=f"Simulert lastprofil {year}",
              labels={"x": "Tidspunkt", "y": "Timeseffekt [kWh/h]"}, template="none")

  return fig_opt_load_profile, total_load_df


def make_fig_load_duration(total_load_df, real_results):
  """Funksjon for å lage varighetskurve"""
  print("opt: ", len(total_load_df))
  print("real", len(real_results["total_load_profile"]))
  total_load_df['real_load_profile'] = real_results["total_load_profile"]
  # ==============================================
  # For optimaliseringsmodellen
  # ==============================================
  # Rearrange in descending order og magnitude
  opt_load_duration_df = total_load_df.sort_values(by=['optimized_load_profile'], ascending=False)

  opt_load_duration_df['interval'] = 1
  # Cumulative sum
  opt_load_duration_df['duration'] = opt_load_duration_df['interval'].cumsum()
  # Percentage of time where the system loadf is greater or equal to the load
  opt_load_duration_df['percentage'] = opt_load_duration_df['duration']*100/len(opt_load_duration_df)


  # ==============================================
  # For reell lastprofil
  # ==============================================
  real_load_duration_df = total_load_df.sort_values(by=['real_load_profile'], ascending=False)

  real_load_duration_df['interval'] = 1
  # Cumulative sum
  real_load_duration_df['duration'] = real_load_duration_df['interval'].cumsum()
  # Percentage of time where the system loadf is greater or equal to the load
  real_load_duration_df['percentage'] = real_load_duration_df['duration']*100/len(real_load_duration_df)

  # =============================================
  # Plot
  # =============================================
  # Opprett en figur
  fig_load_duration = go.Figure()

  # Legg til den første linjen
  fig_load_duration.add_trace(go.Scatter(x=real_load_duration_df['percentage'], y=real_load_duration_df['real_load_profile'], mode='lines', name='Reell lading'))


  # Legg til den andre linjen
  fig_load_duration.add_trace(go.Scatter(x=opt_load_duration_df['percentage'], y=opt_load_duration_df['optimized_load_profile'], mode='lines', name='Optimert lading'))

  # Oppdater layout
  fig_load_duration.update_layout(
      title='Varighetskurve',
      xaxis_title='Prosent av tid [%]',
      yaxis_title='Timeseffekt [kWh/h]',
      template='none',
      width=600,  # Bredden på plottet
      height=500,
  )

  return fig_load_duration


def make_mnd_kostnadsfordeling(months, optimization_results, real_results):
    """Funksjon for å lage figur av månedlig kostnadsfordeling mellom energi-
    og effektkostnader"""
    df_mnd_kostnad = pd.concat(
        [
            pd.DataFrame({'Effektkostnad': real_results["peak_cost_monthly"], 'Energikostnad': real_results["monthly_energy_cost"]}),
            pd.DataFrame({'Effektkostnad': optimization_results["peak_cost_monthly"], 'Energikostnad': optimization_results["monthly_energy_cost"]}),
        ],
        axis=1,
        keys=["Reelle kostnader", "Optimaliserte kostnader"]
    )
    df_mnd_kostnad.index = months

    fig_monthly_cost = _plot_kostnadsfordeling(df_mnd_kostnad, "Måned")

    return fig_monthly_cost


def make_total_kostnadsfordeling(optimization_results, real_results):
    """Funksjon for å lage figur av total kostnadsfordeling mellom energi- og
    effektkostnader"""
    df_tot_kostnad = pd.concat(
        [
            pd.DataFrame({'Effektkostnad': real_results["total_peak_cost"], 'Energikostnad': real_results["total_energy_cost"]}, index=["Totalt"]),
            pd.DataFrame({'Effektkostnad': optimization_results["total_peak_cost"], 'Energikostnad': optimization_results["total_energy_cost"]}, index=["Totalt"]),
        ],
        axis=1,
        keys=["Reelle kostnader", "Optimaliserte kostnader"]
    )
    fig_total_cost = _plot_kostnadsfordeling(df_tot_kostnad, "")
    return fig_total_cost


def _plot_kostnadsfordeling(df, x_axis_title):
  """
  Skal ikke brukes direkte. Brukes av funksjonene for mnd og total
  kostnadsfordeling
  Kilde: https://community.plotly.com/t/grouped-stacked-bar-chart/60805
  """
  # Create a figure with the right layout
  fig = go.Figure(
      layout=go.Layout(
          height=600,
          width=1000,
          barmode="relative",
          xaxis=dict(
              title=x_axis_title  # Tittel på x-aksen
          ),
          yaxis=dict(
              showticklabels=True,
              showgrid=True,
              range=[0, df.groupby(axis=1, level=0).sum().max().max() * 1.05],
              title="Pris [kr]"  # Tittel på y-aksen
          ),
        # Secondary y-axis overlayed on the primary one and not visible
          yaxis2=go.layout.YAxis(
              visible=False,
              matches="y",
              overlaying="y",
              anchor="x",
          ),
          font=dict(size=12),
          legend_x=1.1,
          legend_y=0.5,
          legend_orientation="h",
          hovermode="x",
          margin=dict(b=0,t=40,l=0,r=10),
          template='simple_white',
          title="Fordeling mellom effektkostnader og energikostnader"
      )
  )

  # Define some colors for the product, revenue pairs
  colors = {
      "Reelle kostnader": {
          "Effektkostnad": "#2B6045",
          "Energikostnad": "#5EB88A",
      },
      "Optimaliserte kostnader": {
          "Effektkostnad": "#F28F1D",
          "Energikostnad": "#F6C619",
      }
  }

  # Add the traces
  for i, t in enumerate(colors):
      for j, col in enumerate(df[t].columns):
          if (df[t][col] == 0).all():
              continue
          fig.add_bar(
              x=df.index,
              y=df[t][col],
              # Set the right yaxis depending on the selected product (from enumerate)
              yaxis=f"y{i + 1}",
              # Offset the bar trace, offset needs to match the width
              # For categorical traces, each category is spaced by 1
              offsetgroup=str(i),
              offset=(i - 1) * 1/3,
              width=1/3,
              legendgroup=t,
              legendgrouptitle_text=t,
              name=col,
              marker_color=colors[t][col],
              marker_line=dict(width=2, color="#333"),
              hovertemplate="%{y}<extra></extra>"
          )

  return fig


def make_fig_maks_effekt(optimization_results, real_results, months):
    """
    Lag plot for å vise effekttoppen ved reell og optimert lading
    """
    fig_maks_effekt = px.bar(x=months, y=[real_results["peak_load_monthly"], optimization_results["peak_load_monthly"]], barmode='group',
                            labels={'variable': ''}, template='simple_white')

    # Oppdater layout
    fig_maks_effekt.update_layout(
        title='Månedlig effekttopp',
        xaxis_title='Måned',
        yaxis=dict(
            title='Timeseffekt [kWh/h]',
            showgrid=True  # Aktiverer y-aksens rutenett
        ),
        width=700,  # Bredden på plottet
        height=500,
    )

    # Oppdater legendatitlene
    fig_maks_effekt.for_each_trace(lambda trace: trace.update(name=trace.name.replace("wide_variable_0", "Reell").replace("wide_variable_1", "Optimalisert")))

    return fig_maks_effekt


def make_violin_effektfordeling(optimization_results, real_results, months, start_time_utc):
    """
    Funksjon for å lage violinplot av effektfordeling pr måned for reell og optimert lading
    """
    n_reell = len(real_results["total_load_profile"])
    n_optimalisert = len(optimization_results["total_load_profile"])

    datetime_reell = [start_time_utc + pd.Timedelta(hours=x) for x in range(n_reell)]

    # Datetime-liste for 'Optimalisert' som starter på nytt fra start_time_utc
    datetime_optimalisert = [start_time_utc + pd.Timedelta(hours=x) for x in range(n_optimalisert)]
    datetime_combined = datetime_reell + datetime_optimalisert

    # Opprett en DataFrame for å kombinere begge listene
    df_load_profiles = pd.DataFrame({
        'DateTime': datetime_combined,
        'Effekt': real_results["total_load_profile"] + optimization_results["total_load_profile"],
        'Type': ['Reell'] * len(real_results["total_load_profile"]) + ['Optimalisert'] * len(optimization_results["total_load_profile"])
    })

    df_load_profiles['Month'] = df_load_profiles['DateTime'].dt.month.apply(lambda x: months[x-1])

    # Opprette figuren
    fig_effektfordeling = go.Figure()

    # Legge til violinplot for hver måned og type
    for month in months[:len(df_load_profiles['Month'].unique())]:
        for mode in ['Reell', 'Optimalisert']:
            filtered_df = df_load_profiles[(df_load_profiles['Month'] == month) & (df_load_profiles['Type'] == mode)]
            side = 'negative' if mode == 'Reell' else 'positive'
            color = 'blue' if mode == 'Reell' else 'orange'

            fig_effektfordeling.add_trace(go.Violin(x=filtered_df['Month'],
                                    y=filtered_df['Effekt'],
                                    name=f'{mode} - {month}',
                                    side=side,
                                    line_color=color))

    # Oppdater plottegenskaper
    fig_effektfordeling.update_traces(meanline_visible=True, jitter=0.001)
    fig_effektfordeling.update_layout(violingap=0, violinmode='overlay',
                      title='Fordeling av ladeeffekt - Reell vs Optimalisert',
                      xaxis_title='Måned',
                      yaxis=dict(
                                title='Timeseffekt [kWh/h]',
                                showgrid=True  # Aktiverer y-aksens rutenett
                      ),
                      showlegend=False,
                      width=1200,
                      template="simple_white")

    return fig_effektfordeling


def make_fig_max_power(optimization_results):
    """
    Funksjon for å plotte maksimal effektforbruk for bilene i optimaliseringsmodellen
    """
    max_power = [max(schedule) for schedule in optimization_results['vehicles_charging_schedule'].values()]

    fig_max_power = px.histogram(max_power, template="simple_white", histnorm='percent', nbins=40)

    # Oppdater layout for å inkludere akseltitler og diagramtittel, og for å fjerne legenden
    fig_max_power.update_layout(
        title='Fordeling av maksimal ladeeffekt',
        xaxis_title='Maksimal effekt [kW]',
        yaxis=dict(
            title='Frekvens i prosent',
            showgrid=True  # Aktiverer y-aksens rutenett
        ),
        showlegend=False,
        width=700,  # Bredden på plottet
        height=500,
        template='simple_white'
    )

    return fig_max_power


In [ ]:
#fig2, df = make_violin_effektfordeling(optimization_results, real_results, months, start_time_utc)
#fig2.show()

#df

## Loop

In [ ]:
df_all_scenarios = pd.DataFrame(columns=["Power method", "Year", "Peak tariff", "Total cost real", "Total cost opt",
                                         "Energy cost real", "Energy cost opt", "Peak cost real", "Peak cost opt",
                                         "peak_load_monthly real", "peak_load_monthly opt"])


def make_ny_rad(optimization_results, real_results, power_method, year, peak_tariff_name):
    ny_rad = {"Power method": power_method, "Year": year, "Peak tariff": peak_tariff_name,
              "Total cost real": real_results["total_cost"], "Total cost opt": optimization_results["total_cost"],
              "Energy cost real": real_results["total_energy_cost"], "Energy cost opt": optimization_results["total_energy_cost"],
              "Peak cost real": real_results["total_peak_cost"], "Peak cost opt": optimization_results["total_peak_cost"],
              "peak_load_monthly real": real_results["peak_load_monthly"], "peak_load_monthly opt": optimization_results["peak_load_monthly"]}
    return ny_rad

i = 120000

In [ ]:
#i=50
for power_method in effekt_list:
    for year in year_list:
      for peak_tariff_name in peak_tariff_list:
          # =================================================
          # Variables
          # =================================================
          start_time_utc = pd.to_datetime(f"{year}-01-01 09:00:00")
          start_time_utc_jun = pd.to_datetime(f"{year}-06-01 02:00:00")
          peak_tariff = tariff_mapping.get(peak_tariff_name, ValueError)
          months = ['januar', 'februar', 'mars', 'april', 'mai', 'juni', 'juli', 'august', 'september', 'oktober']

          # ================================
          # Kjør optimaliseringsmodell
          # ================================
          # Simulering 1:Januar - Mai
          opt_dict_jan_may = run_simulation(start_time_utc, peak_tariff, df_overview_csv_jan_mai, df_spotprice_csv, power_method)

          # Simulering 2: Juni - Oktober
          opt_dict_jun_oct = run_simulation(start_time_utc_jun, peak_tariff, df_overview_csv_jun_okt, df_spotprice_csv, power_method)


          # =================================================
          # Reeell lading
          # =================================================
          real_monthly_df, df_overview_shifted, df_detailed_shifted, load_df = get_real_costs(
                        df_overview_csv, df_detailed_csv, df_spotprice_csv, peak_tariff, start_time_utc)

          # Den ønskede sluttdatoen og tiden for DataFrame --> Sørger for at dataene går til samme sluttidspunkt for reell og optimert lading
          #end_datetime = pd.Timestamp(f'{year}-10-30 21:00')

          # Fyller DataFrame med 0-verdier opp til den angitte sluttdatoen og tiden.
          #load_df = load_df.reindex(pd.date_range(load_df.index.min(), end_datetime, freq='H')).fillna(0)
          # Beregn antall rader som trengs for å nå 7261
          rows_to_add = 7261 - len(load_df)

          # Hvis det er rader som må legges til
          if rows_to_add > 0:
              # Opprett en ny indeks som starter fra siste punkt i load_df og øker timevis
              new_index = pd.date_range(start=load_df.index[-1] + pd.Timedelta(hours=1),
                                        periods=rows_to_add,
                                        freq='H')

              # Opprett en ny DataFrame med denne indeksen og fyll med 0
              new_df = pd.DataFrame(index=new_index, columns=load_df.columns).fillna(0)

              # Kombiner de to DataFrames
              load_df = pd.concat([load_df, new_df])



          # ===============================================
          # Lagre resultater
          # ===============================================

          # -------------------------------------------------
          # For optimalisering --> Slå sammen resultatene fra de to halvårene
          # -------------------------------------------------

          # Sørg for at incremental telling fortsetter
          start_key = len(opt_dict_jan_may)
          # Oppdater nøklene i opt_dict_jun_oct
          opt_dict_jun_oct["vehicles_charging_schedule"] = {str(int(key) + start_key): value for key, value in opt_dict_jun_oct["vehicles_charging_schedule"].items()}

          # Lag json-fil
          # (NB! legger til 27 0-er fordi dette er tiden mellom siste observering i første simulering og første observering i andre simulering)
          optimization_results = {
              "peak_tariff": opt_dict_jan_may["peak_tariff"],  # Peak tariff for hele året
              "start_date": start_time_utc.strftime('%Y-%m-%d'),  # Startdato for hele simuleringen
              "total_cost": opt_dict_jan_may["total_cost"] + opt_dict_jun_oct["total_cost"],  # Adderer "total_cost" fra begge halvårene
              "total_energy_cost": opt_dict_jan_may["total_energy_cost"] + opt_dict_jun_oct["total_energy_cost"],  # Adderer "total_energy_cost" fra begge halvårene
              "total_peak_cost": opt_dict_jan_may["total_peak_cost"] + opt_dict_jun_oct["total_peak_cost"],  # Adderer "total_peak_cost" fra begge halvårene
              "exceeded_power": opt_dict_jan_may["exceeded_power"] + opt_dict_jun_oct["exceeded_power"],  # Adderer "exceeded_power" fra begge halvårene
              "peak_load_monthly": {**opt_dict_jan_may["peak_load_monthly"], **opt_dict_jun_oct["peak_load_monthly"]}, # Slår sammen "peak_load_monthly" fra halvårene til en ny dict
              "peak_cost_monthly": {**opt_dict_jan_may["peak_cost_monthly"], **opt_dict_jun_oct["peak_cost_monthly"]}, # Slår sammen "peak_cost_monthly" fra halvårene til en ny dict
              "monthly_energy_use": {**opt_dict_jan_may["monthly_energy_charge"], **opt_dict_jun_oct["monthly_energy_charge"]}, # Slår sammen "monthly_energy_charge" fra halvårene til en ny dict
              "monthly_energy_cost": {**opt_dict_jan_may["monthly_energy_cost"], **opt_dict_jun_oct["monthly_energy_cost"]}, # Slår sammen "monthly_energy_cost" fra halvårene til en ny dict
              "vehicle_energy_cost": opt_dict_jan_may["vehicle_energy_cost"] + opt_dict_jun_oct["vehicle_energy_cost"],  # Kombinerer listene fra halvårene
              "total_load_profile": opt_dict_jan_may["total_load_profile"] + [0]*27 + opt_dict_jun_oct["total_load_profile"],  # Kombinerer listene fra halvårene
              "vehicles_charging_schedule": {**opt_dict_jan_may["vehicles_charging_schedule"], **opt_dict_jun_oct["vehicles_charging_schedule"]}, # Slår sammen "vehicles_charging_schedule" fra halvårene til en ny dict
          }

          # -----------------------------------------------------
          # For reell lading
          # -----------------------------------------------------
          real_results = {
              "peak_tariff": peak_tariff,  # Peak tariff for hele året
              "start_date": start_time_utc.strftime('%Y-%m-%d'),  # Startdato for hele simuleringen
              "total_cost": real_monthly_df["total_cost"].sum(),  # Summerer sammen for alle månedene
              "total_energy_cost": real_monthly_df["energy_cost"].sum(),  # Summerer sammen for alle månedene
              "total_peak_cost": real_monthly_df["Monthly_Peak_Cost"].sum(),
              "peak_load_monthly": real_monthly_df['Peak_load'].to_dict(),
              "peak_cost_monthly": real_monthly_df['Monthly_Peak_Cost'].to_dict(),
              "monthly_energy_use": real_monthly_df['Charged_energy'].to_dict(),
              "monthly_energy_cost": real_monthly_df['energy_cost'].to_dict(),
              #"vehicle_energy_cost": load_df["vehicle_energy_cost"],  # Denne lages senere
              "total_load_profile": load_df["Charged_energy"].tolist(),  # total lastprofil
              #"vehicles_charging_schedule": {**opt_dict_jan_may["vehicles_charging_schedule"], **opt_dict_jun_oct["vehicles_charging_schedule"]}, # Slår sammen "vehicles_charging_schedule" fra halvårene til en ny dict
          }

          # ----------------------------------------------------
          # For sammenligning mellom scenarier
          # ----------------------------------------------------
          ny_rad = make_ny_rad(optimization_results, real_results, power_method, year, peak_tariff_name)
          df_all_scenarios = df_all_scenarios.append(ny_rad, ignore_index=True)

          # =============================================
          # Lagre json-filer
          # =============================================
          if save_json:
            # Lagre data fra optimaliseringen
            with open(f'opt_results_Ncars{len(df_overview_csv_jan_mai)+len(df_overview_csv_jun_okt)}_{power_method}_{year}_{peak_tariff_name}', 'w') as file:
              json.dump(optimization_results, file)

            # Lagre data fra reelle resultater
            with open(f'real_results_{power_method}_{year}_{peak_tariff_name}', 'w') as file:
              json.dump(real_results, file)

          # ==============================================
          # Lag figurer og lagre filer på Google Drive
          # ==============================================
          fig_dict = make_figures(optimization_results, real_results, start_time_utc, months)

          # Dette vil be om autorisasjon for å koble til din Google Drive
          drive.mount('/content/drive')

          # Spesifiser stiene der du vil lagre filen på din Google Drive
          path_base = f"/content/drive/My Drive/Master/Resultater"
          scenario_path = f"/{power_method}/{year}/{peak_tariff_name}"
          path_optimization = f"{path_base+scenario_path}/opt_results_{power_method}_{year}_{peak_tariff_name}.json"
          path_real_json = f"{path_base+scenario_path}/real_results_{power_method}_{year}_{peak_tariff_name}.json"

          # Lagre optimaliseringsresultatene
          with open(path_optimization, 'w') as file:
            json.dump(optimization_results, file)

          # Lagre reelle resultater
          with open(path_real_json, 'w') as file:
            json.dump(real_results, file)

          # Lagre figurer
          for fig_name, fig in fig_dict.items():
            image_path_on_drive = f'{path_base+scenario_path}/{fig_name}_{power_method}_{year}_{peak_tariff_name}.png'
            fig.write_image(image_path_on_drive)

          print(f"Ferdig! ({power_method}, {year}, {peak_tariff_name})")

          # Lagre sammenligning mellom resultater
          df_all_scenarios.to_csv(f"{path_base}/df_all_scenarios{i}.csv")
          i += 1

3590
<class 'int'>
149 days, 14:00:00
start_time 2023-01-01 09:00:00 |, timedelta: 149 days, 14:00:00, dtype:
Set parameter DualReductions to value 0
Set parameter Method to value 0
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


Solve phase I feasrelax model to determine minimal relaxation

Academic license 2427574 - for non-commercial use only - registered to as___@nmbu.no
Optimize a model with 9402190 rows, 9592486 columns and 23984790 nonzeros
Model fingerprint: 0x0b1b9b32
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+02]
Presolve time: 11.93s
Presolved: 206354 rows, 4995035 columns, 995298 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4960083e-01 

<ipython-input-13-ed16aff8ba9c>:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_scenarios = df_all_scenarios.append(ny_rad, ignore_index=True)


opt load profile ok
opt:  7261
real 7261
load duration ok
monthly cost ok
total cost ok
maks effekt ok
violin ok
max power ok
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ferdig! (Reell, 2023, Lik)
3590
<class 'int'>
149 days, 14:00:00
start_time 2022-01-01 09:00:00 |, timedelta: 149 days, 14:00:00, dtype:
Set parameter DualReductions to value 0
Set parameter Method to value 0
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


Solve phase I feasrelax model to determine minimal relaxation

Academic license 2427574 - for non-commercial use only - registered to as___@nmbu.no
Optimize a model with 9402190 rows, 9592486 columns and 23984790 nonzeros
Model fingerprint: 0x0b1b9b32
Coefficient statistics:
  Matrix range     [1e+0

<ipython-input-13-ed16aff8ba9c>:105: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



opt load profile ok
opt:  7261
real 7261
load duration ok
monthly cost ok
total cost ok
maks effekt ok
violin ok
max power ok
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ferdig! (Reell, 2022, Lik)


In [ ]:
# Dette vil be om autorisasjon for å koble til din Google Drive
drive.mount('/content/drive')

# Spesifiser stiene der du vil lagre filen på din Google Drive
path_base = f"/content/drive/My Drive/Master/Resultater"
scenario_path = f"/{power_method}/{year}/{peak_tariff_name}"
path_optimization = f"{path_base+scenario_path}/opt_results_{power_method}_{year}_{peak_tariff_name}.json"
path_real_json = f"{path_base+scenario_path}/real_results_{power_method}_{year}_{peak_tariff_name}.json"

# Lagre optimaliseringsresultatene
with open(path_optimization, 'w') as file:
  json.dump(optimization_results, file)

# Lagre reelle resultater
with open(path_real_json, 'w') as file:
  json.dump(real_results, file)

# Lagre figurer
for fig_name, fig in fig_dict.items():
  image_path_on_drive = f'{path_base+scenario_path}/{fig_name}_{power_method}_{year}_{peak_tariff_name}.png'
  fig.write_image(image_path_on_drive)

print(f"Ferdig! ({power_method}, {year}, {peak_tariff_name})")

# Lagre sammenligning mellom resultater
df_all_scenarios.to_csv(f"{path_base}/df_all_scenarios{i}.csv")
i += 1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ferdig! (Reell, 2022, Lik)


In [ ]:
for fig_name, fig in fig_dict.items():
  print(fig_name)
  fig.show()

fig_opt_load_profile


fig_load_duration


fig_monthly_cost


fig_total_cost


fig_maks_effekt


fig_max_power


In [ ]:
print("hei")

hei


In [ ]:
from datetime import datetime

# Henter nåværende dato og klokkeslett
naa = datetime.now()

# Formatere dato og klokkeslett
dato_og_klokkeslett = naa.strftime("%Y-%m-%d %H:%M:%S")
dato_og_klokkeslett

'2023-12-27 17:00:52'

In [ ]:
df_all_scenarios.to_csv(f"{path_base}/df_all_scenarios{i}.csv")

In [ ]:
print("hei")

hei
